In [16]:
from Env import Env
from RISparser import readris
import pandas as pd
import warnings
from collections import defaultdict
import re
from nltk.tokenize import sent_tokenize,word_tokenize
from sklearn.model_selection import train_test_split
from HAN.HAN_ import parseData,judgeWords,ConvertToid,covid_dataset,HAN_Attention
from torch.utils.data import Dataset
import torch
import numpy as np


In [17]:

## 加载字典
int_vocab  = np.load('../HAN/int_vocab.npy',allow_pickle=True).item()
vocab_int = np.load('../HAN/vocab_int.npy',allow_pickle=True).item()

In [18]:
warnings.filterwarnings('ignore')
TrainingIncludes = readris(open("../DATA/1_Training_Included_20878.ris.txt", 'r', encoding="utf-8"))
TrainingExcludes = readris(open('../DATA/2_Training_Excluded_38635.ris.txt', 'r', encoding="utf-8"))
TrainingDf = parseData(TrainingIncludes, TrainingExcludes)
stopwords = pd.read_csv('../DATA/pubmed.stoplist.csv')
stopwords = stopwords.values.squeeze().tolist()
Train_copy = TrainingDf.copy()
Train_copy['Sen_Num'] = None
MAX_Sen_Len = 18
Max_SEN_Number = 16
MIN_SEN_Number = 3

for i in range(0, len(Train_copy["content"])):
    Train_copy.at[i, "content"] = sent_tokenize(Train_copy.loc[i, "content"])
    Train_copy.loc[i, "Sen_Num"] = len(Train_copy.loc[i, "content"])
    s = []
    for sen in Train_copy.loc[i, "content"]:
        sen = re.sub(r'[.,"\'?:!;=><\\/]', ' ', sen)
        words = word_tokenize(sen)
        cutwords = [word.lower() for word in words if (word.lower() not in stopwords) and judgeWords(word)]
        s.append(cutwords)
    Train_copy.at[i, "content"] = s

Train_copy = Train_copy.query('@MIN_SEN_Number<Sen_Num<@Max_SEN_Number')
Train_copy.reset_index(inplace=True, drop=True)


Data_INT = Train_copy["content"].copy()
for i in range(0, len(Data_INT)):
    Data_INT.at[i] = ConvertToid(vocab_int, Data_INT.iloc[i])

x_train, x_val, y_train, y_val = train_test_split(Data_INT, Train_copy["label"], test_size=0.1, random_state=42)

# loss function
tran_set = covid_dataset(x_train, y_train)
Vali_set = covid_dataset(x_train, y_train)

model = HAN_Attention(len(int_vocab.keys()),100,256,2).cuda()
model.load_state_dict(torch.load("../HAN/HAN.pth"))


<All keys matched successfully>

In [20]:
state = Env.reset()
action = 3
next_state, reward, done = Env.step(action)

tensor(0)
torch.Size([])
tensor([0])
torch.Size([1])
<class 'torch.Tensor'>
tensor([True])
